[![View in Deepnote](https://deepnote.com/static/buttons/view-in-deepnote-white.svg)](https://deepnote.com/viewer/github/katetruman/MultiAgentEC/blob/master/AgentsAndEvents.ipynb)

# Multi-hospital Organ Transplants 

This organ donation scenario contains a monitoring system and individual hospitals as agents, and models some expectations these agents may have, and whether the expectations are fulfilled or violated. 

In this scenario, patients in need of an organ donation can be added to a wait list. When a donor becomes available, the waiting list should be checked for matches with potential recipients. If a match / matches are found, an organ transplant should take place. 

This notebook describes the actions that agents can make. We then explore different expectation rules related to these actions in other notebooks. Please refer to **Expectations.ipynb** for general notes on expectations, and the **ExpectationExamples** folder for specific examples.

## Scenario Agents and Events

In the organ donation scenario and other product tracking scenarios, we have resources and orders which require resources to complete. In the organ donation scenario, our "orders" are the patients on the organ waiting lists, and our "resources" are the organs which are offered for donation. In this scenario, it is the resources that are the limiting factor - we have to wait for a donation offer to be made in order to match a donor and a recipient / recipients. This is contrast to many other scenarios, where we can stockpile our resources but have to wait for an order to be made to know which resources to allocate it at the time. 

An example of such a scenario would be a business which makes furniture. A certain amount of furniture can be made in advance of orders, so we acquire resources and must wait for a customer order in order to move the stock. In the furniture business scenario, we may also be able to accept orders for furniture that has not yet been made. Thus, such a scenario requires lists of both existing resources and orders. In the organ donation scenario, we create a list of organ "orders", but do not need to store organ "resources" for future orders due to the quick turn around required between a donor's death and their organs being transplanted.

The following diagram shows the main events needed to facilitate resource to order matching in a generalised scenario. We will use the furniture store example to explain the meaning of the events. The monitoring system can be thought of as the main agent for a business or organisation.
- The **monitoring system** receives **resource add requests** from a **resource provider** agent. In the case of a furniture store, this could be the store receiving notifications about wood suitable for furnitute making which is available from a particular mill.
- The monitoring system is able to either **accept** or **reject** the **resource add request**. If a request is accepted, the resource list detailing currently available resources will be updated.
- **Customers** can make **product orders** which the monitoring system can **match** with available resources, **reject**, or add to a **waiting list**.
- When a new or waiting order is matched with available resources, the monitoring system sends a match notification to the relevant agents. This could include the **resource** provider, the **customer** and any **supporting agents**. The monitoring system then waits for each of the notified agents to **accept the match**. While there may be certain scenarios where the customer shouldn't need to confirm their order, in this case we assume that they do need to. In the furniture store example, a **supporting agent** could be the courier service for transporting the furniture, and they have to confirm they have the capability to accept a large order before the monitoring system fully processes it.
- Once all agents who have been notified about the match have accepted it (if required), the monitoring system **confirms the match**. After this point, any actions that need to occur for the customer to get their order are undertaken.

If the image below is not displaying properly, you can view it [here](https://github.com/katetruman/MultiAgentEC/blob/master/ActorImages/general.png).
<img src="ActorImages/general.png" width="800">


We now consider the organ transplant specific scenario, in which we have no need for a resource waiting list.
- A hospital with a patient in need of an organ sends a **waiting list add request**. The **monitoring system** can accept or reject it.
- A hospital with a patient ready to donate organs sends a **donor offer** to the monitoring system. The monitoring system either matches the donor offer to one or more patients on the waiting list, or declares that no match could be found. If no match is found, the monitoring system sends a notification to the hospital which made the donation offer.
- If a match is found, the monitoring system notifies the **recipient's hospital**, the **donor's hospital** and the **transplant location**.
- Once all three agents have accepted the match, the monitoring system sends a **match confirmation** message to the agents, and the transplant then takes place.

If the image below is not displaying properly, you can view it [here](https://github.com/katetruman/MultiAgentEC/blob/master/ActorImages/organ.png).
<img src="ActorImages/organ.png" width="800">

### Set up

We load additional predicates from the `dec:notation.pl` file.

In [ ]:
?- cd('~/work'), ['dec:notation'].
?- initialiseDEC.
?- retractall(happensAtNarrative(_,_)).

true.
false.
true.

### Agents

**monSys** (Monitoring System) is our main agent. It facilitates organ transplants by using information about potential recipients and donors to match them. **monSys** receives messages from hospitals (other agents), which detail requests to add patients to organ waiting lists, offer available organs for donation, and respond to previous messages. Both **monSys** and the individual hospitals have expectations about various steps of the organ transplant process.

Note that the first argument of `initiates`, `terminates` and `happensAtNarrative` clauses use the `Agent:Event` format, while the argument of `initially` clauses uses the `Agent:Fluent` format. 

The `agent/1` fluent is used to declare which agents are known to **monSys**. For example, the following code means that from time period zero, **Otago** hospital is recognised as an agent by **monSys**.
- `initially(monSys:agent("Otago")).`

We declare four agents as being recognised by **monSys** at the start of our scenario. They are Otago, Christchurch, Wellington and Auckland hospitals.

In [ ]:
% File: agents.pl
initially(monSys:agent("Otago")).
initially(monSys:agent("Christchurch")).
initially(monSys:agent("Wellington")).
initially(monSys:agent("Auckland")).

### Messaging

In order to show the behaviour of multiple agents, we allow **monSys** to send messages to and receive messages from individual agents. For example, Otago hospital could send a message to **monSys**, asking for a patient to be added to the organ waiting list. After **monSys** has successfully added the patient to the waiting list, **monSys** can send a message which confirms the addition. We wish to model our agents as having distinct events and fluents, and thus messages must be sent in order to share information. A message is received one time period after it is sent. We can use the `causes/3` predicate to declare that one event causes another to occur at the next time period. The `causes`, `send` and `receives` fluents would be better placed in **dec:notation.pl**, but are shown here for clarity. The sending and receiving capability could be used by a wide range of agents, and is not restricted to the organ transplant scenario.

In [ ]:
% File: insertDec.pl
:- discontiguous causes/3.
:- multifile causes/3.
% Provide support for "follow-on" events using causes/3.
happensAt(Hospital2, Event2, T1):- 
    causes(Hospital1:Event1, Hospital2:Event2, T), 
    happensAt(Hospital1, Event1, T), T1 is T + 1.


% Recipient agent receives request one time period after sender agent sends message.
causes(Hospital:send(Recipient, Message), Recipient:receive(Hospital, Message), _).

### Organ waiting list

If a hospital has a patient that they want to be added to the organ waiting list, they send a `waitAddReq` (waiting list add request) message to **monSys**. This message includes the patient's ID, the organs that they need, and a list of further patient details, which could include their sex assigned at birth, ethnicity, date of birth, health conditions and any other notes. 

The general structure of the message is as follows: 
- `happensAtNarrative(Hospital:send(monSys, waitAddReq(ID, Organs, Details)), Time)`. 

For example, the following event indicates that at time period zero, Otago sends a waiting list add request to **monSys** requesting that patient 101 be added to the waiting list for a liver and kidney transplant. The details provided for Patient 101 are that they are female, and their date of birth is the 1st of January 2000.
- `happensAtNarrative("Otago":send(monSys, waitAddReq(101, ["liver", "kidney], [sex:"F", dob:"01/01/2000"])), 0).` 

This type of message could be easily adjusted for a different scenario, with the second argument of a waiting list add request referring to furniture items, medical supplies or lifestock instead of organs.

In order to reduce the number of manual event entries for modelling this scenario, the monitoring system performs several actions automatically. For instance, **monSys** automatically accepts requests to add a patient to the waiting list if the add request does not cause any expectation rule violations. The hospital is notified that a patient has been successfully added by a `waitAccept` message. If any expectation rule violations occur, the request is denied and the hospital is notified by a `waitReject` message. Again, this behaviour could apply to a range of other situations. 


In [ ]:
% File: waitList.pl
% The monitoring system automatically accepts wait list requests if there are no violations.
initiates(monSys:receive(Hospital, waitAddReq(ID, Organs, Details)), waiting(ID, Hospital, Organs, Details, T),T):- 
\+ happensAt(monSys, viol(happ(receive(Hospital, waitAddReq(ID, Organs, Details))),_,_,_),T).

% When a wait list request is accepted or rejected, the hospital is notified one time period afterwards.
causes(monSys:viol(happ(receive(Hospital, waitAddReq(ID, _, _))), _, _, _, _, Reason), 
monSys:send(Hospital, waitReject(ID, Reason)), _).

happensAt(monSys, send(Hospital, waitAccept(ID)), T1):- 
    happensAt(monSys, receive(Hospital, waitAddReq(ID, _, _)), T), 
    \+ happensAt(monSys, viol(happ(receive(Hospital, waitAddReq(ID, _, _))), _, _, _, _, _),T), 
    T1 is T + 1.

A hospital can also request that a patient be removed from the waiting list, by sending a `waitDelReq` (waiting list deletion request) message to **monSys**. This message includes the patientID and a reason for their removal from the list.
For example, the following event indicates that at time two, Otago sends a waiting list deletion request to **monSys** requesting that patient 101 be deleted from the waiting list, due to the death of the patient.
- `happensAtNarrative("Otago":send(monSys, waitDelReq(101, "Patient Death")), 2).`

A patient can also be removed from the waiting list or the list of organs that they are waiting for can be modified, upon a successful organ transplant taking place.

In [ ]:
% File: waitListDeletions.pl

% If a hospital asks that a patient be removed from the waiting list, their "waiting" status is terminated.
terminates(monSys:receive(Hospital, waitDelReq(ID,_)), waiting(ID,Hospital,_,_,_),_).

% If the waiting patient receives organs they needed, remove them from the waiting list.
terminates(monSys:receive(Location, transplantOutcome(MatchID, success)), 
waiting(RecipientID, RecipientHospital,_,_,_), T):- 
    holdsAt(monSys:matched(MatchID, _, _, RecipientHospital, RecipientID, Location, _, _), T).

% Create a new entry for the patient if they are still waiting on any organs.
initiates(monSys:receive(Location, transplantOutcome(MatchID, success)), waiting(RecipientID, 
RecipientHospital,NewOrgans, _, T1), T):- 
    holdsAt(monSys, matched(MatchID, _, _, RecipientHospital, RecipientID, Location, ReceiveOrgans, _), T),
    holdsAt(monSys, waiting(RecipientID, RecipientHospital, OldOrgans, _, T1),T), 
    subtract(OldOrgans, ReceiveOrgans, NewOrgans), NewOrgans \= [].

### Donor offers

A hospital can make a donation offer when a patient becomes eligible for donation by sending a `donorOffer` message to **monSys**. (The eligibility criteria are not covered in this notebook, but in order for a patient to become eligible, brain death or circulatory death must have occurred.) 
The general structure of a `donorOffer` message is:
- `happensAtNarrative(Hospital:send(monSys, donorOffer(PatID, Organs, Details)), Time)`
For example, the following event indicates that at time period two, Wellington sends a donation offer message to **monSys** offering organs for donation from patient 201. The eligible organs are the heart, kidney, liver, lungs and pancreas, and the patient is a Māori male who has a date of birth of 10th of November, 1978.
- `happensAtNarrative("Wellington":send(monSys, donorOffer(201, [heart, kidney, liver, lungs, pancreas], [sex:"M", ethnicity:["Māori"], dob:"10/11/1978"])),2).`

When **monSys** receives a donorOffer, it initiates the **donorOffered\4** fluent.

In [ ]:
% File: donorOffered.pl
initiates(monSys:receive(Hospital, donorOffer(PatID, Organs, Details)), 
donorOffered(Hospital, PatID, Organs, Details), _).

### Matching donors and recipients

We treat **match** events which match a donor with a potential recipient as being manual events - i.e. these are not automatically created by **monSys**, although **monSys** may have expectations about match events. Once a donor offer has been received by **monSys**, if the **noMatch** event occurs, it means that **monSys** did not make a suitable match between the offered donor and a patient on the waiting list. The **noMatch** event contains the ID of the patient from the donation offer, and a reason no match was made. The **noMatch** event causes an **offerRejected** message to be sent to the hospital which made the offer. The rejection message also contains the patient ID and reason for the rejection.

In [ ]:
% File: donationRefusal.pl
causes(monSys:noMatch(Hospital, ID, Reason), monSys:send(Hospital, offerRejected(ID, Reason)), _).

Match events occur for **monSys** when a match is made between a donation offer and a patient on the waiting list. Note that as donor may provide multiple organs, multiple matches may occur from one donation offer. The match includes the donor and the recipient's ID and hospitals, the location allocated to perform the transplant, the organs to be transplanted and any notes which these hospitals should be aware of. Note that the donor's hospital, the recipient's hospital and the transplant location must not all be unique - for instance, the transplant may take place at the recipient's local hospital. If a hospital has more than one role in a match, it will receive communications from **monSys** for each role it undetakes.

The general structure of a match event is as follows:
- `happensAtNarrative(monSys:match(MatchID, DonorHospital, DonorID, RecipientHospital, RecipientID, Location, Organs, Notes), Time)`
For example, the following event indicates that at time period four, the monitoring system performs match 401 between the donation offer for patient 201 from Wellington to the recipient 102 from Otago. The transplant is to take place at Auckland, and the only organ being transplanted is the heart. The transplant notes state that the donor is hepatitis C positive.
- `happensAtNarrative(monSys:match(401, "Wellington", 201, "Otago", 102, "Auckland", [heart], [donorCondition:"HepC"]),4).`

When *monSys* makes a match, it notifies the donor's hospital, the recipient's hospital and the allocated location for the transplant to take place. These hospitals are sent a **recipientFound**, **donorFound** or **locationSelected** message respectively. These events initiate fluents of the same name as the message type. A match event also initiates a **matched** fluent for **monSys**. The fluent has the same arguments as the event.

In [ ]:
% File: matchNotification.pl
causes(monSys:match(MatchID, DonorHospital, DonorID, RecipientHospital, _, Location, Organs, Notes), 
monSys:send(DonorHospital, recipientFound(MatchID, DonorID, RecipientHospital, Location, Organs, Notes)),
 _).

causes(monSys:match(MatchID, DonorHospital, _, RecipientHospital, RecipientID, Location, Organs, Notes), 
monSys:send(RecipientHospital, donorFound(MatchID, RecipientID, DonorHospital, Location, Organs, Notes)),
 _).

causes(monSys:match(MatchID, DonorHospital, _, RecipientHospital, _, Location, Organs, Notes), 
monSys:send(Location, locationSelected(MatchID, DonorHospital, RecipientHospital, Organs, Notes)), _).

initiates(_:receive(monSys, recipientFound(MatchID, DonorID, RecipientHospital, Location, Organs, Notes)), 
recipientFound(MatchID, DonorID, RecipientHospital, Location, Organs, Notes), _).
initiates(_:receive(monSys, donorFound(MatchID, RecipientID, DonorHospital, Location, Organs, Notes)), 
donorFound(MatchID, RecipientID, DonorHospital, Location, Organs, Notes), _).
initiates(_:receive(monSys, locationSelected(MatchID, DonorHospital, RecipientHospital, Organs, Notes)), 
locationSelected(MatchID, DonorHospital, RecipientHospital, Organs, Notes), _).

initiates(monSys:match(MatchID, DonorHospital, DonorID, RecipientHospital, RecipientID, Location, 
Organs, Notes), matched(MatchID, DonorHospital, DonorID, RecipientHospital, RecipientID, Location, 
Organs, Notes), _).

A match event initiates the **role/3** fluent which details the role of each hospital involved in the planned transplant. The general structure of the fluent is *role(Hospital, MatchID, Role)*. The three roles are *donor*, *recipient* and *transplant*, where the final name refers to the hospital where the transplant should take place.

For example, the following fluent represents that Otago is the recipient's hospital for match 401:
- `role("Otago", 401, recipient)`

In [ ]:
% File: matchRoles.pl
initiates(monSys:match(MatchID, DonorHospital, _, _, _, _, _, _), role(DonorHospital, MatchID, donor), _).
initiates(monSys:match(MatchID, _, _, RecipientHospital, _, _, _, _), role(RecipientHospital, MatchID, 
recipient), _).
initiates(monSys:match(MatchID, _, _, _, _, Location, _, _), role(Location, MatchID, transplant), _).

Once a hospital has received a `recipientFound`, `donorFound` or `locationSelected` notification, they can send an `acceptMatch` message to accept their role in the proposed transplant. `acceptMatch` events initiate `matchAccepted` fluents. Once `acceptMatch` messages for all three roles have been received, a `confirmationMatch` event automatically takes place for **monSys**. This event takes the match ID as its sole argument, and causes a `confirmedNotification` message to be sent to the hospitals involved. This message signifies that the hospitals should undertake whatever actions are required for the transplant to take place, such as removing the required organs from the donor, transporting the organs and the recipient to the transplant location, and finally undertaking the transplant.

In [ ]:
% File: matchConfirmation.pl
initiates(monSys:receive(Hospital, acceptMatch(ID)), matchAccepted(Hospital,ID),_).

happensAt(monSys, confirmationMatch(MatchID),T):- 
    holdsAt(monSys, role(DonorHospital, MatchID, donor), T), 
    holdsAt(monSys, role(RecipientHospital, MatchID, recipient), T), 
    holdsAt(monSys, role(Location, MatchID, transplant), T),
    happensAt(monSys, receive(Hospital, acceptMatch(MatchID)),T), (Hospital = DonorHospital ; 
    Hospital = RecipientHospital; Hospital = Location),
    (happensAt(monSys, receive(DonorHospital, acceptMatch(MatchID)),T); 
    holdsAt(monSys, matchAccepted(DonorHospital, MatchID),T)), 
    (happensAt(monSys, receive(RecipientHospital, acceptMatch(MatchID)),T); 
    holdsAt(monSys, matchAccepted(RecipientHospital, MatchID),T)),
    (happensAt(monSys, receive(Location, acceptMatch(MatchID)),T); 
    holdsAt(monSys, matchAccepted(Location, MatchID),T)), !.

initiates(monSys:confirmationMatch(MatchID), confirmed(MatchID), _).

causes(monSys:confirmationMatch(MatchID), monSys:send(Hospital, confirmedNotification(MatchID)), T):- 
    holdsAt(monSys, role(Hospital, MatchID, _),T).

A `transplant` event should take place at the selected transplant location. The `transplant` event has two arguments - the match ID and the outcome of the transplant (e.g. "Success" or "Failure"). In reality, the outcome could be much more complicated e.g. the patient is recovering well but needs to be monitored closely. However, we won't go into this level of detail here. A `transplant` event initiates the `transplanted` fluent, and causes `transplantOutcome` messages to be sent to the recipient hospital, the donor hospital and **monSys**.

In [ ]:
% File: transplant.pl
initiates(_:transplant(ID, Outcome), transplanted(ID, Outcome), _).
causes(Location:transplant(ID, Outcome), Location:send(OtherHospital, transplantOutcome(ID, Outcome)),T):- 
    holdsAt(Location, locationSelected(ID, DonorHospital, RecipientHospital, _, _), T), 
    (OtherHospital = DonorHospital ; OtherHospital = RecipientHospital; OtherHospital = monSys).

### Statistics requests

As well as taking part in matching active patients, **monSys** and hospitals may have an interest in viewing statistics on historical data.

In [ ]:
% File: statRequests.pl
%happensAtNarrative("Wellington":statReq(), 15).

## Saving the scenario

So that we can use the events and fluents declared above to support a range of different expectation rules, we save the contents of the above cell blocks into a file we can consult. The kernel automatically saves the contents (excluding comments) of all the cells in a notebook to a file in the **output_files** folder when the notebook is run. Because this will be overwritten next time a notebook is run, we copy the file to our current directory.

In [ ]:
% PYTHON
import shutil
shutil.copyfile("/work/output_files/allCells.pl", "/work/AE.pl")

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=32f94018-a4da-40ef-8c9f-8983d73811c8' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>